# Exploratory Data Analysis With SQLite

## INTRODUCE DATABASE
In this project, we'll work with data from the [CIA World Factbook](https://www.cia.gov/the-world-factbook/), a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like the following:
- **population:** the global population.
- **population_growth:** the annual population growth rate, as a percentage.
- **area:** the total land and water area.

[Download database here](https://dsserver-prod-resources-1.s3.amazonaws.com/257/factbook.db)

## 1.Load Database

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [2]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


## 2.Overview of The Data

In [3]:
%%sql
SELECT *
FROM facts
LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


- **name**: the name of the country.
- **area**: the country's total area (both land and water).
- **area_land**: the country's land area in square kilometers.
- **area_water**: the country's waterarea in square kilometers.
- **population**: the country's population.
- **population_growth**: the country's population growth as a percentage.
- **birth_rate**: the country's birth rate, or the number of - births per year per 1,000 people.
- **death_rate**: the country's death rate, or the number of death per year per 1,000 people.

## 3.Summary Statistics

### 3.1.Which country has the highest population? And Population Growth ?

In [4]:
%%sql
SELECT MIN(population) AS "Min Population",
        MAX(population) AS "Max Population",
        MIN(population_growth) AS "Min Population Growth",
        MAX(population_growth) AS "Max Population Growth"
FROM facts        

Done.


Min Population,Max Population,Min Population Growth,Max Population Growth
0,7256490011,0.0,4.02


A few things stick out from the summary statistics::
 - There is a country population ~ 0
 - There is a country population ~ 7.2 bilion

In [5]:
%%sql
SELECT name, population, population_growth
FROM facts
WHERE population == (SELECT MAX(population) FROM facts)
        OR population_growth == (SELECT MAX(population_growth) FROM facts)
        OR population == (SELECT MIN(population) FROM facts)

Done.


name,population,population_growth
South Sudan,12042910,4.02
Antarctica,0,None
World,7256490011,1.08


There are two countries that have special values:
- **Antarctica:** According to Wikipedia, Antarctica has no permanent inhabitants. It seems like the table contains a row for Antarctica, which explains the population of 0
- **World:** the row contains the whole world, which explains the maximum population of over 7.2 billion we found earlier

> ***We should calculate the summary statistics we calculated earlier while excluding the row for the whole world.***

In [6]:
%%sql
SELECT  CASE
        WHEN population == (SELECT MAX(population) 
                                   FROM facts
                                   WHERE name != 'World') THEN "The Highest Population: " || name
        WHEN population_growth == (SELECT MAX(population_growth) 
                                   FROM facts
                                   WHERE name != 'World') THEN "The Highest Population Growth: "|| name 
        ELSE NULL
        END AS 'COUNTRY', population, population_growth
FROM facts
ORDER BY COUNTRY DESC
LIMIT 2

Done.


COUNTRY,population,population_growth
The Highest Population: China,1367485388,0.45
The Highest Population Growth: South Sudan,12042910,4.02


- **South Sudan** has the highest population growth
- **China** has the highest population

### 3.2.Which countries have the highest ratios of water to land? 

In [7]:
%%sql
SELECT  name , area_water, area_land, MAX(ROUND(CAST(area_water AS FLOAT)/area_land,2)) AS 'ratios_water_to_land'
FROM facts

Done.


name,area_water,area_land,ratios_water_to_land
British Indian Ocean Territory,54340,60,905.67


**British Indian Ocean Territory** has the highest ratios of water to land

### 3.3.Which countries have more water than land?

In [8]:
%%sql
SELECT name, area_water, area_land
FROM facts
WHERE area_water > area_land 
    OR (area_water NOT NULL AND area_land IS NULL)
ORDER BY area_water DESC

Done.


name,area_water,area_land
Ethiopia,104300,None
British Indian Ocean Territory,54340,60
Virgin Islands,1564,346


There are three countries that have more water than land are **Ethiopia**, **British Indian Ocean Territory**, and **Virgin Islands**.

### 3.4.Which countries will add the most people to their populations next year?

In [9]:
%%sql
SELECT name, migration_rate 
FROM facts
WHERE migration_rate == (SELECT MAX(migration_rate)
                         FROM facts
                         WHERE name != 'World')

Done.


name,migration_rate
Qatar,22.39


With migration rate ~ 22.39 (highest), **Qatar** will add the most people to their populations next year  

### 3.5.Which countries have a higher death rate than birth rate?

In [10]:
%%sql
SELECT name, death_rate,  birth_rate
FROM facts
WHERE death_rate > birth_rate AND name != 'World'
ORDER BY death_rate

Done.


name,death_rate,birth_rate
Monaco,9.24,6.65
Austria,9.42,9.41
Japan,9.51,7.93
Saint Pierre and Miquelon,9.72,7.42
Bosnia and Herzegovina,9.75,8.87
Italy,10.19,8.74
Poland,10.19,9.74
Czech Republic,10.34,9.63
Portugal,11.02,9.27
Greece,11.09,8.66



## 4.Finding Densely Populated Countries

### 4.1.The Densely Populated Countries have:
- **Above** average values for **population**.
- **Below** average values for **area**.

In [11]:
%%sql
SELECT ROUND(AVG(population)) AS 'Average Population', ROUND(AVG(area)) AS 'Average Area'
FROM facts
WHERE name != 'World'

Done.


Average Population,Average Area
32242667.0,555094.0


In [12]:
%%sql
SELECT name, population, area
FROM facts
WHERE population > (SELECT AVG(population)
                   FROM facts
                   WHERE name != 'World')
AND  area < (SELECT AVG(area)
                   FROM facts
                   WHERE name != 'World')

Done.


name,population,area
Bangladesh,168957745,148460
Germany,80854408,357022
Iraq,37056169,438317
Italy,61855120,301340
Japan,126919659,377915
"Korea, South",49115196,99720
Morocco,33322699,446550
Philippines,100998376,300000
Poland,38562189,312685
Spain,48146134,505370


### 4.2.Which countries have the highest population/area ratio?

In [13]:
%%sql
SELECT name, population, area, ROUND(CAST(population AS FLOAT)/area,2) AS Population_Density
FROM facts
WHERE area NOT NULL AND population NOT NULL
        AND area != 0 AND population != 0
ORDER BY Population_Density DESC;

Done.


name,population,area,Population_Density
Macau,592731,28,21168.96
Monaco,30535,2,15267.5
Singapore,5674472,697,8141.28
Hong Kong,7141106,1108,6445.04
Gaza Strip,1869055,360,5191.82
Gibraltar,29258,6,4876.33
Bahrain,1346613,760,1771.86
Maldives,393253,298,1319.64
Malta,413965,316,1310.02
Bermuda,70196,54,1299.93


**THERE IS A DIFFERENCE BETWEEN 4.1 AND 4.2**

## REFERENCES
- [Dataquest.io](https://github.com/dataquestio/solutions/blob/master/Mission257Solutions.ipynb)